# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-07-22 to 2021-07-22
You have 435 stocks
['A', 'AAP', 'ABBV', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BSX', 'BWA', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CFG', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DPZ', 'DRE', '

ticker,A,AAP,ABBV,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,...,WY,WYNN,XEL,XLNX,XOM,XYL,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-07-16 00:00:00+00:00,148.93,205.85,117.50,317.98,117.51,311.9100,606.10,160.44,58.21,205.60,...,34.02,104.70,69.13,128.63,57.32,119.360,153.40,518.33,49.60,199.68
2021-07-19 00:00:00+00:00,147.58,205.67,115.42,314.62,118.79,309.1800,602.05,158.98,57.45,201.68,...,32.95,104.74,68.27,129.33,55.35,117.230,150.17,506.91,47.70,197.83
2021-07-20 00:00:00+00:00,149.30,210.23,116.05,323.59,118.73,312.0600,608.72,160.91,58.03,203.14,...,33.65,106.57,68.16,131.10,55.96,121.530,155.63,519.67,50.20,200.11
2021-07-21 00:00:00+00:00,148.95,209.40,116.99,322.85,118.98,313.7700,612.26,164.84,58.10,204.47,...,34.08,109.59,67.06,135.33,57.76,122.430,159.57,533.77,51.64,198.04
2021-07-22 00:00:00+00:00,150.33,209.60,117.39,322.22,118.30,314.2128,623.11,161.60,57.49,203.04,...,34.12,107.54,67.00,136.07,57.24,122.065,158.25,535.54,50.32,200.48


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAPL'].tail()

,,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day
date,ticker,,,,,


In [6]:
all_factors.describe()

,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day
count,4.380450e+05,4.380450e+05,4.380450e+05,4.380450e+05,4.380450e+05
mean,-1.419317e-19,-3.852433e-20,-1.176006e-19,1.703181e-19,-7.704866e-20
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728084e+00,-1.728079e+00,-1.728074e+00,-1.728074e+00,-1.728074e+00
25%,-8.680186e-01,-8.680186e-01,-8.680186e-01,-8.680186e-01,-8.680186e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.680186e-01,8.680186e-01,8.680186e-01,8.680186e-01,8.680186e-01
max,1.728084e+00,1.728079e+00,1.728074e+00,1.728079e+00,1.728079e+00


In [7]:
all_factors.head()

momentum_252_day_logret  \
date                      ticker                            
2017-07-24 00:00:00+00:00 A                      0.780423   
                          AAP                   -1.680298   
                          ABBV                  -0.015927   
                          ABMD                   0.573372   
                          ABT                    0.087598   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2017-07-24 00:00:00+00:00 A                                  -0.477810   
                          AAP                                 1.680298   
                          ABBV                               -0.493737   
                          ABMD                                0.406138   
                          ABT                                -0.382248   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2017-07-24 00:00:00+00:00 A                                 1.098959   
                          AAP                              -0.605224   
                          ABBV                             -0.629114   
                          ABMD                              1.258229   
                          ABT                              -0.398174   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2017-07-24 00:00:00+00:00 A                         -0.199087   
                          AAP                        1.473247   
                          ABBV                       0.533554   
                          ABMD                      -1.290087   
                          ABT                       -0.007963   

                                  annualzed_volatility_120_day  
date                      ticker                                
2017-07-24 00:00:00+00:00 A                           0.278722  
                          AAP                         1.608626  
                          ABBV                        0.390211  
                          ABMD                        1.306014  
                          ABT                         0.254832

In [8]:
all_factors.tail()

momentum_252_day_logret  \
date                      ticker                            
2021-07-22 00:00:00+00:00 XYL                    1.083032   
                          ZBH                   -0.772457   
                          ZBRA                   1.441389   
                          ZION                   0.692822   
                          ZTS                    0.087598   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2021-07-22 00:00:00+00:00 XYL                                -1.130813   
                          ZBH                                 1.242302   
                          ZBRA                               -0.987471   
                          ZION                                1.130813   
                          ZTS                                -1.321937   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-07-22 00:00:00+00:00 XYL                               0.222977   
                          ZBH                              -0.477808   
                          ZBRA                              0.629114   
                          ZION                             -0.899872   
                          ZTS                               1.019325   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2021-07-22 00:00:00+00:00 XYL                       -1.003398   
                          ZBH                        0.167233   
                          ZBRA                      -0.899872   
                          ZION                       0.437991   
                          ZTS                       -1.027288   

                                  annualzed_volatility_120_day  
date                      ticker                                
2021-07-22 00:00:00+00:00 XYL                        -1.664366  
                          ZBH                        -0.485772  
                          ZBRA                        1.170631  
                          ZION                        1.513060  
                          ZTS                        -0.079635